**Import des bibliothèques nécessaires**

In [35]:
import pandas as pd
import spacy

**Machine learning : Import du fichier généré pour effectuer les tests**

In [36]:
df = pd.read_csv("../data/nlp_generated_text.csv")

In [37]:
df.head()

,texte,sujet
0,"Je suis blessé, j’ai besoin d’un centre médica...",santé
1,Je cherche un centre de santé à Paris où je po...,santé
2,"J’ai des pensées suicidaires, je suis à avenue...",santé
3,Y a-t-il un hôpital ou un médecin proche de ru...,santé
4,Je me suis blessé et je saigne. Où aller à Par...,santé


**Machine learning : Nettoyage du fichier**

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texte   2000 non-null   object
 1   sujet   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [39]:
# On vérifie si la colonne contient des doublons et on les traite s'il y en a.

filtered_df = df[df['texte'].isna()]
print(filtered_df)

df = df.dropna()

df.info()



Empty DataFrame
Columns: [texte, sujet]
Index: []
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   texte   2000 non-null   object
 1   sujet   2000 non-null   object
dtypes: object(2)
memory usage: 31.4+ KB


In [40]:
# On vérifie qu'on a bien que des valeurs avec un type String

print(df['texte'].apply(type).value_counts()) 

texte
<class 'str'>    2000
Name: count, dtype: int64


In [ ]:
# Réaliser le / les modèles de machine learning
# Le but : identifier le sujet concerné par le texte envoyé par l'utilisateur.
# Ce sujet identifié nous permettra de cibler la bonne table à requêter + nous servira aussi dans la partie NLP pour nommé ce sujet.

**API Chatbot**

**NLP : Chargement du modèle français de spaCy**

In [50]:
nlp = spacy.load("fr_core_news_sm")  # Chargement du modèle français


**NLP : Réception d'un texte en entrée**

In [51]:
# Etant dans un notebook Jupyter, je ne peux pas utiliser un input() pour le moment. Il faudra attendre de transférer le code dans un fichier python .py
# received_text = input(Bonjour ! Que puis-je faire pour vous ?")

received_text = "Je cherche les horaires d’une maraude dans Paris, si possible près de rue de Robin."
print ("vous : ", received_text)


vous :  Je cherche les horaires d’une maraude dans Paris, si possible près de rue de Robin.


In [ ]:
# Je veux que "Rue de Robin" soit reconnu comme une localisation. Actuellement "Robin" est détecté comme étant une personne.
# On utilise donc le composant de spaCy nommé EntityRuler por spécifier et prioriser des règles d'entités.
from fastapi import FastAPI
from spacy.pipeline import EntityRuler
app = FastAPI()

ruler = nlp.add_pipe("entity_ruler", before="ner")
 # On crée les patterns à identifier comme étant une localisation. 
 
patterns = [
   
        {
            "label": "LOC",
            "pattern": [
                {"LOWER": {"IN": ["rue", "avenue", "boulevard", "place"]}},
                {"LOWER": {"IN": ["de", "d'", "du", "des"]}},
                {"IS_ALPHA": True, "OP": "+"}  # pour détecter un ou plusieurs mots alphabétiques
            ]
    },
        {
        "label": "LOC",  # Identifier comme une localisation
        "pattern": [{"SHAPE": "ddddd"}]  # Reconnaître les séquences de 5 chiffres
    }
 ]
ruler.add_patterns(patterns)



@app.post("/analyze/")
def analyze_text(text: str):
 
 spacy_doc = nlp(received_text)

 for token in spacy_doc : 
    print(f"{token.text.lower()} → lemma: {token.lemma_.lower()}, POS: {token.pos_}")
    
 for ent in spacy_doc.ents:
    print(f"{ent.text} → label: {ent.label_}")

 lieux = [ent.text for ent in spacy_doc.ents if ent.label_ in ["GPE", "LOC", "FAC"]]
 print("LIEUX DÉTECTÉS :", lieux)

 clean_text = " ".join([
    token.lemma_.lower() for token in spacy_doc
    if not token.is_stop and not token.is_punct
])
 print("Texte nettoyé :", clean_text)

 # En attendant d'avoir réalisé l'algorithme de machine learning permettant de classer le texte de l'utilisateur comme étant "santé", 
 # "judiciaire", "maraude" ou "douche", on simule un sujet = "santé" pour la suite du text.
 sujet = "santé"
 # sujet = classifier_subject_model.predict(text)


 if len(lieux) == 1:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}")
 elif len(lieux) == 2:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}, {lieux[1]}")
 elif len(lieux) == 3:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}, {lieux[1]}, {lieux[2]}")
 elif len(lieux) == 4:
    print(f"Vous vous intéressé au sujet {sujet} sur la localisation {lieux[0]}, {lieux[1]}, {lieux[2]}, {lieux[3]}")
 else :
    print(f"Vous vous intéressé au sujet {sujet}. Pouvez-vous nous donner le code postal, s'il vous plait ?")


 return classifier(text)


# Lancer avec : uvicorn script:app --reload



IndentationError: unexpected indent (1996645456.py, line 22)

In [ ]:
# Si pas de code postal dans "lieux" alors le demander


In [ ]:
# pour la suite, je pensais trouver un moyen de matcher la / les localisations trouvées avec un code postal.
# Ce code postal pourrait nous permettre de proposer les hopitaux, commissariats, maraudes, douches (en fonction du sujet concerné) 
# dans le code postal (via requête BDD / table concernée)

# Si pas de localité ou localité insuffisante pour matcher un code postal, on pourrait proposer les deux premiers chiffres des 8 départements de L'ile de france,
# en fonction du choix réalisé, on pourrait proposer la list de CP, et en fonction du choix, la liste des hopitaux, commissariats, 
# maraudes, douches (en fonction du sujet concerné).

# Pour la liste des des CP et informations , on pourrait scrapper des sites (tel que : https://www.hopital.fr/annuaire/Paris+Ile-de-France+75/) 
# ou réaliser un fichier nous mêmes avec de fausses informations pour simuler les données ou utiliser une API disponible pour récupérer ces infos

In [67]:
# Pas sûre de garder cette partie sur TfidfVectorizer. On a trop de lignes et trop de caractères par ligne pour pouvoir voir quelque chose j pense

#from sklearn.feature_extraction.text import TfidfVectorizer

#vectorizer = TfidfVectorizer()
#X = vectorizer.fit_transform(df["clean_text"])

In [68]:
#print(vectorizer.get_feature_names_out())  # Liste des mots

In [69]:
#print(X.toarray()[10:30])  # Matrice TF-IDF sur un échantillon

In [70]:
""""
from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")
 
result = classifier("J'adore ce produit, il est excellent !")
print(result)

"""


'"\nfrom transformers import pipeline\nclassifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")\n\nresult = classifier("J\'adore ce produit, il est excellent !")\nprint(result)\n\n'